In [13]:
from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import utils
from utils import plots, get_batches, plot_confusion_matrix, get_data


from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

%matplotlib inline

pd.set_option('display.max_columns', 100)

In [2]:
PROJECT_DIR  = '/home/ubuntu/Extracting-food-preferences'
PICTURES_DIR = '/home/ubuntu/Extracting-food-preferences/classification_with_pics'

In [11]:
PROJECT_DIR  = 'C:\\Users\\Natalia\\Documents\\GitHub\\Extracting-food-preferences'
PICTURES_DIR = 'C:\\Users\\Natalia\\Documents\\GitHub\\Extracting-food-preferences\\classification_with_pics'

In [9]:
%cd $PROJECT_DIR

C:\Users\Natalia\Documents\GitHub\Extracting-food-preferences


In [7]:
#import modules
from utils import *
from vgg16 import Vgg16
from datetime import datetime

Using Theano backend.


In [ ]:
%cd $PICTURES_DIR

#Set path to sample/ path if desired
path = PICTURES_DIR #+ '/sample'
#path = PICTURES_DIR + '\\' + 'sample\\'
results_path=path + '/results/'
train_path=path + '/train/'
valid_path=path + '/valid/'

In [28]:
%cd $PICTURES_DIR

#Set path to sample/ path if desired
path = PICTURES_DIR + '\\sample\\'
#path = PICTURES_DIR + '\\' + 'sample\\'
results_path=path + '\\results\\'
train_path=path + '\\train\\'
valid_path=path + '\\valid\\'

model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

C:\Users\Natalia\Documents\GitHub\Extracting-food-preferences\classification_with_pics


In [16]:
#import Vgg16 
vgg = Vgg16()
model = vgg.model

In [17]:
#Set constants
batch_size=25
no_of_epochs=1

In [22]:
# Use batch size of 1 since we're just doing preprocessing on the CPU
val_batches = get_batches(path+'valid', shuffle=False, batch_size=1)
batches = get_batches(path+'train', shuffle=False, batch_size=1)

Found 50 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [23]:
import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]

In [24]:
val_data = get_data(path+'valid')
trn_data = get_data(path+'train')

Found 50 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [26]:
trn_data.shape

(200, 3, 224, 224)

In [27]:
trn_data[0, :,:,:]

array([[[  98.,   98.,  103., ...,   86.,   78.,   73.],
        [  98.,   98.,  103., ...,   86.,   78.,   73.],
        [ 109.,  109.,  109., ...,   99.,   92.,   88.],
        ..., 
        [ 215.,  215.,  215., ...,  191.,  191.,  191.],
        [ 215.,  215.,  215., ...,  191.,  191.,  191.],
        [ 215.,  215.,  215., ...,  192.,  192.,  192.]],

       [[  75.,   75.,   80., ...,  100.,   92.,   87.],
        [  75.,   75.,   80., ...,  100.,   92.,   87.],
        [  86.,   86.,   86., ...,  112.,  105.,  101.],
        ..., 
        [ 218.,  218.,  218., ...,  198.,  198.,  198.],
        [ 218.,  218.,  218., ...,  198.,  198.,  198.],
        [ 218.,  218.,  218., ...,  199.,  199.,  199.]],

       [[  44.,   44.,   49., ...,  113.,  105.,  100.],
        [  44.,   44.,   49., ...,  113.,  105.,  100.],
        [  55.,   55.,   55., ...,  128.,  121.,  117.],
        ..., 
        [ 223.,  223.,  223., ...,  208.,  208.,  208.],
        [ 223.,  223.,  223., ...,  208., 

In [29]:
save_array(model_path+'train_data.bc', trn_data)
save_array(model_path+'valid_data.bc', val_data)

In [30]:
trn_data = load_array(model_path+'train_data.bc')
val_data = load_array(model_path+'valid_data.bc')

In [31]:
trn_data.shape

(200, 3, 224, 224)

In [32]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

In [33]:
val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

In [36]:
val_batches.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [37]:
trn_features = model.predict(trn_data, batch_size=batch_size)
val_features = model.predict(val_data, batch_size=batch_size)

In [38]:
trn_features.shape

(200, 1000)

In [39]:
save_array(model_path+'train_lastlayer_features.bc', trn_features)
save_array(model_path+'valid_lastlayer_features.bc', val_features)

In [40]:
trn_features = load_array(model_path+'train_lastlayer_features.bc')
val_features = load_array(model_path+'valid_lastlayer_features.bc')

In [41]:
# 1000 inputs, since that's the saved features, and 2 outputs, for dog and cat
lm = Sequential([ Dense(2, activation='softmax', input_shape=(1000,)) ])
lm.compile(optimizer=RMSprop(lr=0.1), loss='categorical_crossentropy', metrics=['accuracy'])

In [45]:
batch_size=20

In [48]:
lm.fit(trn_features, trn_labels, nb_epoch=5, batch_size=batch_size, 
       validation_data=(val_features, val_labels))

Train on 200 samples, validate on 50 samples
Epoch 1/5
200/200 [==============================] - 0s - loss: 0.3179 - acc: 0.8450 - val_loss: 1.6171 - val_acc: 0.5400
Epoch 2/5
200/200 [==============================] - 0s - loss: 0.3041 - acc: 0.8450 - val_loss: 1.6038 - val_acc: 0.5000
Epoch 3/5
200/200 [==============================] - 0s - loss: 0.3037 - acc: 0.8500 - val_loss: 1.6130 - val_acc: 0.5200
Epoch 4/5
200/200 [==============================] - 0s - loss: 0.3074 - acc: 0.8400 - val_loss: 1.5825 - val_acc: 0.5800
Epoch 5/5
200/200 [==============================] - 0s - loss: 0.2991 - acc: 0.8450 - val_loss: 1.6553 - val_acc: 0.5400


In [44]:
lm.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_10 (Dense)                 (None, 2)             2002        dense_input_1[0][0]              
Total params: 2,002
Trainable params: 2,002
Non-trainable params: 0
____________________________________________________________________________________________________
